In [ ]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE

In [ ]:
from numpy.random import seed
from random import randrange

In [ ]:
df = pd.read_excel("SP100_vol.xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)
train = df.iloc[:int(len(df['NKE'])*0.7)]
validation_length=int(len(df['NKE'])*0.7)-int(len(df['NKE'])*0.5)
train=train.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [ ]:
n_inputs = [3,5,10,15,21,42,84]
mlp_units = [[[712, 712], [712, 712]],[[512, 512], [512, 512]],[[250, 250], [250, 250]],[[100, 100], [100, 100]]]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
n_blocks=[[1, 1],[2, 2],[3, 3],[5, 5]]
dropout_prob_theta=[0, 0.2, 0.3, 0.4]
n_pool_kernel_size=[[2, 2, 1], [4, 2, 1], [4, 4, 2]]
pooling_mode=["MaxPool1d", "AvgPool1d"]
interpolation_mode=['linear', 'nearest']

# Test

In [ ]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [ ]:
model = NHITS(h=5, input_size=21,
                loss=losses[3],
                scaler_type='standard',
                learning_rate=learning_rate[0],
                stack_types=['identity','identity'],
                n_blocks=[2,2],
                mlp_units= [[712, 712], [712, 712]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
                max_steps=150,
                dropout_prob_theta = 0,
                n_pool_kernel_size=[4,2,1],
                pooling_mode='MaxPool1d' ,
                interpolation_mode='nearest',
              random_seed=123422336)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=5,static_df=df_static,n_windows=None, test_size=test_length-test_length%5,step_size=5)
forecasts = forecasts.dropna()
if "NHITS-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["NHITS"].values
else:
  Y_hat=forecasts["NHITS-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 123422336


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.0047935969298586855 | QLIKE: 0.04557353812205509 | MAE: 0.0028944732412508802 | MAPE: 0.21524246305132116


In [ ]:
if "NHITS-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","NHITS-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

In [14]:
df2

NHITS-median                                                    \
unique_id          AAPL       ABT       ACN      ADBE       ADP      AMGN   
ds                                                                          
2009-07-01     0.005120  0.007375  0.005938  0.005770  0.005614  0.005955   
2009-07-02     0.005118  0.007462  0.006071  0.005762  0.005574  0.005900   
2009-07-06     0.005073  0.007426  0.006032  0.005695  0.005642  0.005958   
2009-07-07     0.005334  0.007518  0.006074  0.005908  0.005775  0.006026   
2009-07-08     0.005212  0.007424  0.006123  0.005926  0.005813  0.006016   
...                 ...       ...       ...       ...       ...       ...   
2013-07-18     0.007968  0.008536  0.009279  0.010081  0.006873  0.009420   
2013-07-19     0.007907  0.008534  0.009256  0.010000  0.006828  0.009374   
2013-07-22     0.007754  0.008473  0.009465  0.010040  0.006756  0.009550   
2013-07-23     0.007962  0.008514  0.008834  0.010539  0.007020  0.009857   
2013-07-24     0.008082  0.008591  0.008911  0.010465  0.007269  0.009668   

                                                    ...                      \
unique_id        AMT      AMZN       AXP        BA  ...       TJX       TMO   
ds                                                  ...                       
2009-07-01  0.007012  0.005862  0.007316  0.006213  ...  0.006793  0.006208   
2009-07-02  0.006997  0.005934  0.007373  0.006212  ...  0.006782  0.006206   
2009-07-06  0.007060  0.005911  0.007304  0.006119  ...  0.006791  0.006181   
2009-07-07  0.007039  0.006035  0.007325  0.006189  ...  0.006913  0.006338   
2009-07-08  0.007003  0.005938  0.007256  0.006271  ...  0.006985  0.006230   
...              ...       ...       ...       ...  ...       ...       ...   
2013-07-18  0.007469  0.009852  0.008735  0.011028  ...  0.010661  0.010025   
2013-07-19  0.007408  0.009741  0.008729  0.010971  ...  0.010823  0.009859   
2013-07-22  0.007356  0.009574  0.008587  0.010945  ...  0.010872  0.009794   
2013-07-23  0.007570  0.009232  0.008795  0.011120  ...  0.010653  0.010181   
2013-07-24  0.007719  0.009393  0.008867  0.011114  ...  0.010861  0.010225   

                                                                        \
unique_id        TXN       UNH       UNP       UPS       USB        VZ   
ds                                                                       
2009-07-01  0.007999  0.007150  0.007347  0.005977  0.007890  0.006639   
2009-07-02  0.008139  0.007212  0.007318  0.005886  0.008042  0.006705   
2009-07-06  0.008038  0.007274  0.007380  0.005972  0.007994  0.006696   
2009-07-07  0.007837  0.007542  0.007545  0.006010  0.008252  0.006617   
2009-07-08  0.008165  0.007511  0.007518  0.006041  0.008303  0.006451   
...              ...       ...       ...       ...       ...       ...   
2013-07-18  0.008199  0.008190  0.007811  0.008617  0.011032  0.004892   
2013-07-19  0.008212  0.008088  0.007835  0.008500  0.011369  0.004831   
2013-07-22  0.008339  0.008127  0.007812  0.008851  0.011147  0.004918   
2013-07-23  0.008665  0.008149  0.007887  0.008712  0.010979  0.005044   
2013-07-24  0.008619  0.008101  0.007905  0.008862  0.010855  0.004998   

                                
unique_id        WFC       WMT  
ds                              
2009-07-01  0.009080  0.005774  
2009-07-02  0.008998  0.005761  
2009-07-06  0.009034  0.005762  
2009-07-07  0.009202  0.005785  
2009-07-08  0.009225  0.005834  
...              ...       ...  
2013-07-18  0.013498  0.006026  
2013-07-19  0.013563  0.006083  
2013-07-22  0.013403  0.006144  
2013-07-23  0.012973  0.006078  
2013-07-24  0.013323  0.006184  

[1014 rows x 80 columns]

In [ ]:
df2.to_excel("NHITS forecasts.xlsx")